# Clustering avec k-means

Dans ce cours, nous allons utiliser un des algorithmes d'apprentissage non-supervisé. Les algorithmes non-supervisés n'utilisent pas de cible (label) ; au lieu de cela, leur objectif est d'apprendre certaines propriétés des données, de représenter la structure des caractéristiques d'une certaine manière.
Le clustering signifie simplement l'attribution de points de données à des groupes en fonction de la similitude des points les uns par rapport aux autres. Il est important de se rappeler que cette fonctionnalité de cluster est catégorique.

Il existe de nombreux algorithmes de clustering. Ils diffèrent principalement par la manière dont ils mesurent la « similitude » ou la « proximité » et par les types de fonctionnalités avec lesquels ils fonctionnent. L'algorithme que nous utiliserons, k-means, est intuitif et facile à appliquer. Selon votre application, un autre algorithme peut être plus approprié.

L'algorithme **K-means** mesure la similarité en utilisant la distance euclidienne. Il crée des clusters en plaçant un certain nombre de points, appelés centroïdes, à l'intérieur de l'espace des caractéristiques. Chaque point de l'ensemble de données est affecté au cluster du centroïde le plus proche. Le "k" dans "k-means" est le nombre de centroïdes (c'est-à-dire de clusters) qu'il crée. Vous définissez le k vous-même (c'est un paramètre à passer à l'algorithme).

Prenez le temps de regarder la documentation du k-means dans scikit learn, surtout la liste des paramètres (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

Examinons comment l'algorithme k-means apprend les clusters. Nous nous concentrerons sur trois paramètres de l'implémentation de scikit-learn : *n_clusters, max_iter et n_init*.

C'est un processus simple en deux étapes. L'algorithme commence par initialiser au hasard un nombre prédéfini (n_clusters) de centroïdes. Il itère ensuite sur ces deux opérations :

    1- attribuer des points au centre de gravité du cluster le plus proche
    2- déplacer chaque centroïde pour minimiser la distance à ses points

Il itère sur ces deux étapes jusqu'à ce que les centroïdes ne bougent plus, ou jusqu'à ce qu'un nombre maximum d'itérations soit écoulé (max_iter).

Il arrive souvent que la position aléatoire initiale des centroïdes se termine par un mauvais regroupement. Pour cette raison, l'algorithme se répète un certain nombre de fois (n_init) et renvoie le clustering qui a la plus petite distance totale entre chaque point et son centroïde, le clustering optimal.

Vous devrez peut-être augmenter le *max_iter* pour un grand nombre de clusters ou *n_init* pour un ensemble de données complexe. Ordinairement, le seul paramètre que vous devrez choisir vous-même est *n_clusters* (c'est-à-dire *k*).

## Exemple - Logement en Californie

En tant que caractéristiques spatiales, la « latitude » et la « longitude » de California Housing sont des candidats naturels pour le regroupement des k-moyennes. Dans cet exemple, nous allons les regrouper avec « MedInc » (revenu médian) pour créer des segments économiques dans différentes régions de Californie.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

df = pd.read_csv("../Data/data2/FE-Course-Data/housing.csv")
X = df.loc[:, ["MedInc", "Latitude", "Longitude"]]
X.head()

Étant donné que le clustering k-means est sensible à l'échelle, il peut être judicieux de redimensionner ou de normaliser les données avec des valeurs extrêmes. Nos fonctionnalités sont déjà à peu près à la même échelle, nous les laisserons donc telles quelles.

In [ ]:
# Create cluster feature
kmeans = KMeans(n_clusters=6)
X["Cluster"] = kmeans.fit_predict(X)
X["Cluster"] = X["Cluster"].astype("category")

X.head()

Examinons maintenant quelques *plots* pour voir à quel point cela a été efficace. Tout d'abord, un nuage de points qui montre la répartition géographique des grappes. Il semble que l'algorithme ait créé des segments distincts pour les zones à revenu élevé sur les côtes.

In [ ]:
sns.relplot(
    x="Longitude", y="Latitude", hue="Cluster", data=X, height=6,
);

## Exercice

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)


def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score


# Prepare data
df = pd.read_csv("../Data/data2/FE-Course-Data/ames.csv")

L'algorithme k-means est sensible à l'échelle. Cela signifie que nous devons réfléchir à la manière et à l'opportunité de redimensionner nos fonctionnalités, car nous pourrions obtenir des résultats très différents en fonction de nos choix. En règle générale, si les caractéristiques sont déjà directement comparables (comme un résultat de test à des moments différents), alors vous ne voudriez pas redimensionner. D'un autre côté, les fonctionnalités qui ne sont pas à des échelles comparables (comme la taille et le poids) bénéficieront généralement d'une remise à l'échelle. Parfois, le choix ne sera pas clair cependant. Dans ce cas, vous devriez essayer de faire preuve de bon sens, en vous rappelant que les caractéristiques avec des valeurs plus élevées seront pondérées plus fortement.

### 1) Scaling Features
Considérez les ensembles de fonctionnalités suivants. Pour chacun, décidez si :

   - ils devraient certainement être redimensionnés,
   - ils ne doivent absolument pas être redimensionnés, ou
   - soit peut être raisonnable
   
Features:

   1. `Latitude` and `Longitude` of cities in California
   2. `Lot Area` and `Living Area` of houses in Ames, Iowa
   3. `Number of Doors` and `Horsepower` of a 1989 model car

###  2) Create a Feature of Cluster Labels

Création d'un clustering k-means avec les paramètres suivants :

   - features: `LotArea`, `TotalBsmtSF`, `FirstFlrSF`, `SecondFlrSF`, `GrLivArea`
   - number of clusters: 10
   - iterations: 10

In [ ]:
X = df.copy()
y = X.pop("SalePrice")


# YOUR CODE HERE: Define a list of the features to be used for the clustering
features = ____


# Standardize
X_scaled = X.loc[:, features]
X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)


# YOUR CODE HERE: Fit the KMeans model to X_scaled and create the cluster labels
kmeans = KMeans(____, random_state=0)
X["Cluster"] = ____

Vous pouvez exécuter cette cellule pour voir le résultat du regroupement, si vous le souhaitez.

In [ ]:
Xy = X.copy()
Xy["Cluster"] = Xy.Cluster.astype("category")
Xy["SalePrice"] = y
sns.relplot(
    x="value", y="SalePrice", hue="Cluster", col="variable",
    height=4, aspect=1, facet_kws={'sharex': False}, col_wrap=3,
    data=Xy.melt(
        value_vars=features, id_vars=["SalePrice", "Cluster"],
    ),
);

La méthode `score_dataset` évaluera votre modèle XGBoost avec cette nouvelle fonctionnalité ajoutée aux données d'entraînement.

In [ ]:
score_dataset(X, y)